In [2]:

from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_from_disk
import numpy as np
from processingDatasetDotless import *
import torch
from transformers import pipeline

In [3]:
dataset = load_from_disk("/Users/ammar/Developer/git-repos/dotless/Models/Model v3/AR-multi-dotted-Small-arrow")


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Masked', 'Options', 'Target'],
        num_rows: 500000
    })
})

In [5]:
dataset['train'][20]

{'Masked': 'يوسف الذين دفنوا السلاح و المهمات العسكرية في [MASK] من جب بالعاصمة',
 'Options': ['آكثر',
  'آكثز',
  'آكتر',
  'آكتز',
  'آكبر',
  'آكبز',
  'إكثر',
  'إكثز',
  'إكتر',
  'إكتز',
  'إكبر',
  'إكبز',
  'أكثر',
  'أكثز',
  'أكتر',
  'أكتز',
  'أكبر',
  'أكبز',
  'اكثر',
  'اكثز',
  'اكتر',
  'اكتز',
  'اكبر',
  'اكبز'],
 'Target': 'أكثر'}

In [6]:
indicies = []
for i in range(10000):
    length = len(dataset['train'][i]["Options"])
    if length > 10 and length <= 20:
        indicies.append(i)

print(indicies)
print(len(indicies))
dataset['train'][indicies[3]]


[13, 14, 28, 41, 57, 66, 92, 102, 118, 119, 122, 123, 126, 130, 133, 134, 152, 164, 174, 178, 179, 188, 189, 210, 216, 223, 237, 258, 263, 269, 270, 272, 278, 306, 320, 323, 330, 340, 348, 352, 356, 363, 396, 404, 428, 438, 443, 444, 451, 466, 473, 490, 497, 498, 502, 505, 510, 514, 520, 523, 529, 531, 537, 538, 540, 541, 554, 566, 569, 575, 587, 588, 598, 601, 607, 609, 613, 625, 626, 632, 640, 662, 664, 674, 677, 679, 688, 690, 692, 702, 709, 722, 723, 727, 728, 745, 755, 761, 767, 774, 782, 796, 798, 799, 804, 805, 818, 831, 832, 836, 846, 851, 853, 868, 878, 894, 895, 902, 923, 931, 944, 956, 965, 968, 993, 1001, 1012, 1015, 1033, 1043, 1049, 1057, 1060, 1062, 1070, 1082, 1095, 1103, 1109, 1115, 1117, 1118, 1125, 1126, 1142, 1151, 1155, 1159, 1165, 1169, 1177, 1196, 1197, 1200, 1204, 1205, 1210, 1212, 1216, 1234, 1241, 1245, 1251, 1253, 1260, 1275, 1286, 1290, 1302, 1310, 1316, 1328, 1330, 1331, 1332, 1337, 1338, 1341, 1343, 1351, 1362, 1364, 1368, 1377, 1387, 1394, 1398, 1403, 142

{'Masked': 'السؤال كيف توفق بين دخلك والمنصرفات [MASK] الله كريم فخلص إلى أن',
 'Options': ['ؤةي',
  'ؤةى',
  'ؤةئ',
  'ؤهي',
  'ؤهى',
  'ؤهئ',
  'وةي',
  'وةى',
  'وةئ',
  'وهي',
  'وهى',
  'وهئ'],
 'Target': 'وهي'}

In [7]:
with open("/Users/ammar/Developer/git-repos/dotless/data/arwiki.wordlist", 'r') as file:
    wordlist = set(file.read().split())


In [311]:
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]


In [10]:
pipe = pipeline("fill-mask", model="UBC-NLP/ARBERTv2")

Some weights of the model checkpoint at UBC-NLP/ARBERTv2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
pipe("اللغة [MASK] هي لغة العرب")


[{'score': 0.706984281539917,
  'token': 2134,
  'token_str': 'العربية',
  'sequence': 'اللغة العربية هي لغة العرب'},
 {'score': 0.05566714331507683,
  'token': 5194,
  'token_str': 'التركية',
  'sequence': 'اللغة التركية هي لغة العرب'},
 {'score': 0.04787074029445648,
  'token': 11755,
  'token_str': 'الكردية',
  'sequence': 'اللغة الكردية هي لغة العرب'},
 {'score': 0.01563572697341442,
  'token': 4181,
  'token_str': 'الفرنسية',
  'sequence': 'اللغة الفرنسية هي لغة العرب'},
 {'score': 0.012686226516962051,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': 'اللغة هي لغة العرب'}]

In [49]:
test_wordTT = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("أحمد"))
test_wordTT

[3219]

In [50]:
test_wordTid = tokenizer.convert_ids_to_tokens(test_wordTT)
test_wordTid

['أحمد']

In [315]:
def get_candidate_word_probabilities(input_text, candidate_words):
    tokenized_text = tokenizer.tokenize(input_text)
    masked_word_index = tokenized_text.index('[MASK]')
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze_(0)

    with torch.no_grad():
        output = model(input_ids)

    predictions = output.logits[0, masked_word_index].softmax(dim=0)

    # Tokenize and convert all candidate words to ids at once

    tokenized_candidate_words = [tokenizer.tokenize(word)for word in candidate_words]
    #print(tokenized_candidate_words)
    
    tokenized_candidate_words = [word for word in tokenized_candidate_words if len(word) == 1]
    #print(tokenized_candidate_words)
    
    flattened_tokenized_candidate_words = [word for sublist in tokenized_candidate_words for word in sublist]
    #print(flattened_tokenized_candidate_words)
    
    candidate_word_ids = [tokenizer.convert_tokens_to_ids(TCW) for TCW in flattened_tokenized_candidate_words]
    #print(candidate_word_ids)
    
    # Use a list comprehension to populate the dictionary
    candidate_probabilities = {word: predictions[word_id].item() for word, word_id in zip(flattened_tokenized_candidate_words, candidate_word_ids)}
    #print(candidate_probabilities)

    return candidate_probabilities


def generate_probabilties_end(example):
    input_text = example["Masked"]
    candidates = example["Options"]

            
    word_probabilities = get_candidate_word_probabilities(input_text, candidates)

    sorted_words = sorted(word_probabilities, key=word_probabilities.get, reverse=True)
    if len(sorted_words ) > 0:
        most_probable_word = sorted_words[0]
    else:
        most_probable_word = None
        print(example["Target"])
        
    return word_probabilities, sorted_words, most_probable_word


In [318]:
num_eg = 0
#example = dataset['train'][indicies[num_eg]]
example = mask_random_word("قرأ أحمد كتابه ثم نام")
word_probabilities, sorted_words, most_probable_word = generate_probabilties_end(example)
print("Length of words:", len(sorted_words))
for word in sorted_words:
    probability = word_probabilities[word]
    print(f"Word: '{word}', Probability: {probability:.10f}")
    
print()

print("Most probable word:", most_probable_word)
print("Target word:", example["Target"])
print("------------------------------------------")

found = False
for i in range(len(sorted_words)):
    if sorted_words[i] == example["Target"]:
        print("Sucess at probability level:", i)
        found = True
        break
if not found: print("Not found.")
sucess_level = i

example

ValueError: empty range for randrange() (0, 0, 0)

In [316]:

count = np.zeros(1000000, dtype =int)
#longWords = []
#veryLongWords = []
#veryVeryLongWords = []
start = 0
end = 101
iterationCount = end - start - 1
for i in range(start, end):
    num_eg = i
    #example1 = mask_random_word("التحدي الصعب يكمن في فهم التفاصيل الدقيقة لهذه الفكرة المعقدة")
    #example1 = dataset['train'][indicies[num_eg]]
    example1 = dataset['train'][num_eg]

    word_probabilities, sorted_words, most_probable_word = generate_probabilties_end(example1)
    for word in sorted_words:
        probability = word_probabilities[word]
        
    #if len(sorted_words) >= 500 and len(sorted_words) < 1000: longWords.append(example1["Target"])
    #elif len(sorted_words) >= 1000 and len(sorted_words) < 100000: veryLongWords.append(example1["Target"])
    #elif len(sorted_words) >= 100000 and len(sorted_words) <1000000: veryVeryLongWords.append(example1["Target"])
    #elif len(sorted_words) >= 1000000: veryVeryLongWords.append(example1["Target"])



    for j in range(len(sorted_words)):
        if sorted_words[j] == example1["Target"]:
            count[j] += 1
            break
        

for k in range(len(count)):
    numberCount = count[k]
    if numberCount != 0:
        print(numberCount, "placments at level", k, "  \t", (numberCount/iterationCount) * 100, "%")
        
print()
print("Number of examples tested:", iterationCount)
print("Number of examples that yielded a result:", np.sum(count))


الزملا
تزينت
تجربتها
بنجران
بوشناق
ودلائل
ناروتو
الفندقية
المئوية
التصفية
أدخله
المتلاحقة
بالرباط
لمنهج
لآثار
وشعرا
نيئن
إخطاره
المزيدكشفت
والسياح
جيتيت
ذويه
المطرقة
75 placments at level 0   	 75.0 %
2 placments at level 1   	 2.0 %

Number of examples tested: 100
Number of examples that yielded a result: 77
